In [1]:
texts = [
    "O time venceu a partida na prorrogação!",  # Esportes
    "A bolsa de valores fechou em alta hoje.",  # Economia
    "Desfile de moda apresenta as novas tendências do inverno.",  # Moda
    "O governo anunciou novas medidas econômicas.",  # Economia
    "O campeonato de futebol está emocionante nesta temporada.",  # Esportes
    "Nova coleção de roupas sustentáveis é lançada.",  # Moda
]

labels = [0, 1, 2, 1, 0, 2]  # 0: Esportes, 1: Economia, 2: Moda


In [2]:
%pip install torch torchvision torchaudio
%pip install transformers


  Using cached torchvision-0.21.0-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached torchaudio-2.6.0-cp311-cp311-win_amd64.whl.metadata (6.7 kB)
Using cached torchvision-0.21.0-cp311-cp311-win_amd64.whl (1.6 MB)
Using cached torchaudio-2.6.0-cp311-cp311-win_amd64.whl (2.5 MB)



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\pedro parentoni\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\pedro parentoni\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Nome do modelo em português
modelo_nome = "neuralmind/bert-base-portuguese-cased"

# Número de classes (3: Esportes, Economia, Moda)
num_classes = 3

# Carregar o tokenizer
tokenizer = AutoTokenizer.from_pretrained(modelo_nome, do_lower_case=False)

# Carregar o modelo para classificação
model = AutoModelForSequenceClassification.from_pretrained(modelo_nome, num_labels=num_classes)


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

C:\Users\pedro parentoni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pedro parentoni\.cache\huggingface\hub\models--neuralmind--bert-base-portuguese-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import torch

# Tokenizar os textos
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Converter rótulos para tensores
labels_tensor = torch.tensor(labels)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [5]:
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW

# Criar dataset e dataloader
dataset = TensorDataset(inputs["input_ids"], inputs["attention_mask"], labels_tensor)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Definir otimizador
optimizer = AdamW(model.parameters(), lr=5e-5)

# Mover modelo para GPU se disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loop de treinamento
epochs = 20
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        # Zerar gradientes
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Época {epoch + 1}, Perda: {total_loss:.4f}")



C:\Users\pedro parentoni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Época 1, Perda: 3.4808
Época 2, Perda: 3.0992
Época 3, Perda: 2.2949
Época 4, Perda: 1.7618
Época 5, Perda: 1.1620
Época 6, Perda: 0.8322
Época 7, Perda: 0.4554
Época 8, Perda: 0.3289
Época 9, Perda: 0.2159
Época 10, Perda: 0.1434
Época 11, Perda: 0.1133
Época 12, Perda: 0.1016
Época 13, Perda: 0.0750
Época 14, Perda: 0.0575
Época 15, Perda: 0.0495
Época 16, Perda: 0.0424
Época 17, Perda: 0.0365
Época 18, Perda: 0.0383
Época 19, Perda: 0.0334
Época 20, Perda: 0.0331


In [10]:
# Texto de teste
novos_textos += [
    "O Banco Central elevou a taxa de juros para conter a inflação.",
    "Empresas de tecnologia lideram alta no mercado de ações.",
    "O desemprego recuou 2% no último trimestre, segundo o IBGE.",
    "FMI revisa projeção de crescimento global para baixo.",
    "Governo anuncia pacote de estímulos para pequenas empresas.",
    "Investidores estrangeiros aumentam participação na bolsa brasileira.",
    "Criptomoedas registram volatilidade após regulamentação na Europa.",
    "Novo imposto sobre grandes fortunas entra em debate no Congresso.",
    "Empresas adotam inteligência artificial para otimizar processos financeiros.",
    "Gigante do varejo online reporta lucro recorde no terceiro trimestre.",
    "Setor imobiliário registra alta nas vendas de imóveis de luxo.",
    "País registra superávit comercial acima das expectativas.",
    "Aumento no preço do petróleo pressiona a inflação global.",
    "Reforma tributária avança no Senado após meses de negociação.",
    "Startup fintech capta R$ 200 milhões em nova rodada de investimentos."
]

# Lista de rótulos verdadeiros (substitua pelos rótulos reais)
rotulos_reais = ["Economia"] * len(novos_textos)  # Todos os textos são de economia

# Tokenizar
inputs_teste = tokenizer(novos_textos, padding=True, truncation=True, return_tensors="pt").to(device)

# Rodar no modelo
model.eval()
with torch.no_grad():
    outputs = model(**inputs_teste)

# Obter rótulos previstos
predicoes = torch.argmax(outputs.logits, dim=1).cpu().numpy()

# Mapear rótulos para categorias
categorias = ["Esportes", "Economia", "Moda"]
resultados = [categorias[p] for p in predicoes]

# Calcular número de acertos
acertos = sum(1 for p, r in zip(resultados, rotulos_reais) if p == r)
total = len(novos_textos)
precisao = acertos / total * 100  # Porcentagem de acertos

# Mostrar resultados
for texto, categoria in zip(novos_textos, resultados):
    print(f"Texto: {texto} → Categoria prevista: {categoria}")

print(f"\nNúmero de acertos: {acertos}/{total} ({precisao:.2f}%)")

Texto: O time anunciou um novo treinador. → Categoria prevista: Esportes
Texto: As bolsas de valores caíram drasticamente. → Categoria prevista: Economia
Texto: Nova coleção de verão foi apresentada. → Categoria prevista: Moda
Texto: O valor do Bitcoin caiu 5% → Categoria prevista: Economia
Texto: Trump anuncia medidas protecionistas para o Estado Americano → Categoria prevista: Economia
Texto: O Banco Central elevou a taxa de juros para conter a inflação. → Categoria prevista: Economia
Texto: Empresas de tecnologia lideram alta no mercado de ações. → Categoria prevista: Economia
Texto: O desemprego recuou 2% no último trimestre, segundo o IBGE. → Categoria prevista: Economia
Texto: FMI revisa projeção de crescimento global para baixo. → Categoria prevista: Economia
Texto: Governo anuncia pacote de estímulos para pequenas empresas. → Categoria prevista: Economia
Texto: Investidores estrangeiros aumentam participação na bolsa brasileira. → Categoria prevista: Economia
Texto: Criptomoeda